# Lab 3: Prompt Engineering

In this lab, we explore different prompting strategies to enhance interactions with foundation models.

We will experiment the following approaches: 
1. Basic Prompting – Learn how to craft clear and effective prompts.
2. Role-Based Prompting – Use system prompts to align responses with specific roles.
3. In-Context Learning (Few-Shot Prompting) – Enhance model performance by providing relevant examples.
4. Chain-of-Thought (CoT) Prompting – Implement step-by-step reasoning strategies.
5. ReAct Prompting with Web Search – Combine reasoning and action-driven prompting to retrieve real-time information.

Note: Different GenAI models have unique approaches to writing prompts. To get the best results, refer to their official documentation for prompt templates designed for specific use cases:
- **OpenAI**: [Prompt Examples](https://platform.openai.com/docs/examples)
- **Llama 3**: [Prompt Formats](https://www.llama.com/docs/model-cards-and-prompt-formats/meta-llama-3/)

Throughout this lab, we will analyze how each technique influences AI-generated responses.

## 1\. Set up the enviroment

In [1]:
import os
from dotenv import load_dotenv
from IPython.display import Markdown

from langchain_openai import ChatOpenAI
from langchain_together import ChatTogether
from langchain.schema import SystemMessage, HumanMessage
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.tools import tool
from langchain_community.tools import TavilySearchResults


In [2]:
# Load required API keys
load_dotenv()

openai_api_key = os.getenv("OPENAI_API_KEY")
together_api_key = os.getenv("TOGETHER_API_KEY")
tavily_api_key = os.getenv("TAVILY_API_KEY")

In [3]:
# Initialize the GPT-4o model as the base model
llm_gpt = ChatOpenAI(
    model_name="gpt-4o",
    temperature=0.7, # We use a higher temperature
    max_retries=2,
    timeout= 120, 
    openai_api_key=openai_api_key)

# Initialize the DeepSeek-R1 as the reasoning model
llm_deepseek_r1 = ChatTogether(
    model="deepseek-ai/DeepSeek-R1",
    max_retries=2, 
    timeout= 120,
    together_api_key= together_api_key
)

## 2\. Benchmark Travel Plan  

This is a well-planned and practical travel itinerary. It maximizes your time while ensuring a relaxed and enjoyable road trip through the Southwest.

The plan has been tested and proven effective, making it a great benchmark for comparing AI-generated itineraries.


In [4]:
# This is a human-created travel plan
human_created_plan = """
### **7-Day Southwest Road Trip Itinerary: Los Angeles → Grand Canyon → Page → Monument Valley → Zion → Las Vegas → Los Angeles**  

This road trip takes you through some of the most breathtaking landscapes in the American Southwest, combining iconic national parks, scenic drives, and vibrant city experiences.  

---

## **Day 1: Los Angeles → Las Vegas (5-hour drive)**  
- Depart from **Los Angeles** at **1:00 PM**.  
- Drive **4-5 hours** to **Las Vegas**, arriving in the early evening.  
- Check into the hotel and drop off luggage.  
- Enjoy dinner at a top-rated restaurant.  
- Stroll along the **Las Vegas Strip**, visit famous casinos, or catch a fountain show at **Bellagio**.  

 **Overnight in Las Vegas**  

---

## **Day 2: Las Vegas → Grand Canyon National Park (5-hour drive)**  
- Breakfast in Las Vegas.  
- Depart for **Grand Canyon National Park (South Rim)**, stopping for fuel and snacks along the way.  
- Arrive in the afternoon and explore nearby **viewpoints and trails**:  
  - **Yavapai Geology Museum** for panoramic views.  
  - Short walks along the **Rim Trail**.  
- Check into a hotel inside the park.  
- Dinner at **Yavapai Tavern**, known for its **Elk Burger**.  
- Head to **Mather Point** for a stunning **sunset view**.  
- Return to the hotel, rest, and stargaze if the sky is clear.  

**Overnight in Grand Canyon National Park**  

---

## **Day 3: Grand Canyon → Page (Antelope Canyon & Horseshoe Bend) (4-hour drive)**  
- Wake up early for **sunrise at Hopi Point**, one of the best vantage spots in the park.  
- Breakfast at the hotel.  
- Depart **by 9:00 AM** for **Page, Arizona** (**2-3 hour drive**).  
- Arrive in **Page** and take a **guided tour of Antelope Canyon** (**Upper or Lower**, depending on availability).  
- Lunch at **BirdHouse**, famous for its crispy fried chicken.  
- Check into the hotel and take a short rest.  
- Visit **Horseshoe Bend** in the late afternoon for the **sunset**, capturing the dramatic curves of the Colorado River.  
- Return to the hotel and rest.  

 **Overnight in Page**  

---

## **Day 4: Page → Monument Valley (5-hour drive)**  
- Breakfast in Page.  
- Depart for **Monument Valley**, stopping at **Forrest Gump Point** for a classic photo opportunity.  
- Lunch near **Monument Valley Tribal Park**.  
- Drive the **17-mile Monument Valley Scenic Loop**, taking in the awe-inspiring rock formations (**2-hour self-guided drive**).  
- Check into **The View Hotel**, the best location for panoramic views.  
- Relax and enjoy **sunset views over Monument Valley** from the hotel balcony.  

**Overnight at The View Hotel, Monument Valley**  

---

## **Day 5: Monument Valley → Lake Powell → Zion National Park (5-hour drive)**  
- Watch the **sunrise from The View Hotel**, one of the most breathtaking sunrises in the Southwest.  
- Breakfast at the hotel.  
- Drive **2-3 hours** to **Lake Powell**, making scenic stops along the way.  
- Quick lunch before heading to **Zion National Park** (**2-hour drive**).  
- En route, stop at **Pine Creek Canyon Overlook** for a **1-hour round-trip hike** with incredible canyon views.  
- Check into a hotel near Zion.  
- Relax before heading out for dinner.  

**Overnight near Zion National Park**  

---

## **Day 6: Zion National Park → Las Vegas (2-3 hour drive)**  
- Breakfast at the hotel.  
- Drive **2-3 hours** back to **Las Vegas**.  
- Spend the afternoon exploring Las Vegas, shopping, or relaxing at a resort.  
- Catch a show at **The Sphere**, a one-of-a-kind immersive experience.  
- Return to the hotel and rest.  

 **Overnight in Las Vegas**  

---

## **Day 7: Las Vegas → Los Angeles (4-5 hour drive)**  
- Breakfast at the hotel.  
- Depart for **Los Angeles** (**4-5 hour drive**).  
- Arrive in **Los Angeles** in the afternoon.  

**Trip ends!**  

---

## **Travel Tips & Considerations**  
**Book accommodations in advance**, especially for Grand Canyon and Monument Valley.  
**Arizona enforces strict speed limits**, so be mindful of patrols.  
**Limit daily driving to a maximum of 5-6 hours**, always during daylight for safety.  
**Purchase a National Park Annual Pass** to save on entry fees for Grand Canyon and Zion.  
**Plan meal stops in advance**, as dining options in remote areas can be limited.  

---

"""


## 3\. Basic Prompting  

Let's look at a basic prompt without any advanced techniques — just a simple, natural question that feels human. 

In [5]:
# A simple prompt does not adhere to many prompt engineering prinples

simple_prompt = """
Create a 7-day road trip  from Los Angeles that goes through the Southwest
"""

response1 = llm_gpt.invoke(simple_prompt)

display(Markdown(response1.content))

A 7-day road trip from Los Angeles through the Southwest can be an incredible adventure, offering a mix of natural wonders, vibrant cities, and cultural sites. Here's a suggested itinerary:

### Day 1: Los Angeles to Joshua Tree National Park
- **Distance:** ~130 miles
- **Highlights:**
  - Start your journey early and drive to Joshua Tree.
  - Explore the unique desert landscapes and rock formations.
  - Visit Hidden Valley and Barker Dam trails.
  - Overnight in or near the park.

### Day 2: Joshua Tree to Sedona, Arizona
- **Distance:** ~350 miles
- **Highlights:**
  - Depart for Sedona, known for its red rock formations.
  - On the way, stop at the Route 66 town of Kingman for a nostalgic break.
  - Arrive in Sedona and take a scenic drive through Oak Creek Canyon.
  - Explore local shops and art galleries.
  - Overnight in Sedona.

### Day 3: Sedona to Grand Canyon National Park
- **Distance:** ~115 miles
- **Highlights:**
  - Head to Grand Canyon South Rim.
  - Spend the day exploring viewpoints like Mather Point and Yavapai Point.
  - Consider a short hike along the Rim Trail or part of the Bright Angel Trail.
  - Overnight in Grand Canyon Village or nearby.

### Day 4: Grand Canyon to Page, Arizona
- **Distance:** ~140 miles
- **Highlights:**
  - Depart for Page, stopping at Desert View Watchtower and Little Colorado River Gorge.
  - Visit Horseshoe Bend for stunning views of the Colorado River.
  - Explore Antelope Canyon (book a guided tour in advance).
  - Overnight in Page.

### Day 5: Page to Monument Valley, Utah
- **Distance:** ~125 miles
- **Highlights:**
  - Drive to Monument Valley, renowned for its iconic sandstone buttes.
  - Take a guided tour of the valley to learn about Navajo culture and history.
  - Enjoy sunset views and photograph the stunning landscape.
  - Overnight in Monument Valley or nearby.

### Day 6: Monument Valley to Flagstaff, Arizona
- **Distance:** ~200 miles
- **Highlights:**
  - Head south to Flagstaff, stopping at sites like Navajo National Monument.
  - Explore Flagstaff's historic downtown and visit the Lowell Observatory.
  - Consider a side trip to Walnut Canyon National Monument.
  - Overnight in Flagstaff.

### Day 7: Flagstaff to Los Angeles
- **Distance:** ~400 miles
- **Highlights:**
  - Begin your return journey to Los Angeles.
  - Optional stop at the Petrified Forest National Park if time permits.
  - Enjoy a relaxing drive back, reminiscing about your adventure.

### Tips:
- **Accommodation:** Book accommodations in advance, especially in popular areas like the Grand Canyon and Monument Valley.
- **Vehicle:** A reliable vehicle is essential; consider renting an SUV for comfort and better handling in diverse terrains.
- **Weather:** Check weather conditions, especially during winter months, as some areas can experience snow and road closures.
- **Permits:** Some sites, like Antelope Canyon, require permits or guided tours.

This itinerary allows you to experience a mix of natural beauty and cultural richness, creating memories that will last a lifetime.

In [ ]:
# Let's refine it with clear instuctions

basic_prompt = """
I want to plan a road trip starting from Los Angeles that goes through all these amazing places in the Southwest - Grand Canyon, Page, Monument Valley, Zion National Park, and Las Vegas. 
We'll be away for 7 days and need to leave LA after lunch on the first day and get back to LA in the afternoon on the last day.
"""

response1 = llm_gpt.invoke(basic_prompt)

display(Markdown(response1.content))

That sounds like an amazing road trip! Here's a suggested itinerary to make the most of your 7-day adventure through the Southwest, starting and ending in Los Angeles:

### Day 1: Los Angeles to Grand Canyon
- **Afternoon Departure:** Leave Los Angeles after lunch.
- **Drive:** Head towards Grand Canyon South Rim (~7.5 hours, 480 miles).
- **Evening:** Check into a hotel or campsite near the South Rim. Relax and prepare for the next day's exploration.

### Day 2: Explore Grand Canyon
- **Morning:** Spend the day exploring the Grand Canyon. Visit popular viewpoints like Mather Point, Yavapai Observation Station, and Desert View.
- **Activities:** Consider a hike along the Rim Trail or take a guided tour.
- **Evening:** Overnight stay near the Grand Canyon.

### Day 3: Grand Canyon to Page
- **Morning:** Depart for Page, Arizona (~2.5 hours, 130 miles).
- **Activities:** Visit Horseshoe Bend and take a tour of Antelope Canyon (book in advance).
- **Evening:** Explore Lake Powell or relax in Page. Overnight in Page.

### Day 4: Page to Monument Valley
- **Morning:** Drive to Monument Valley (~2 hours, 125 miles).
- **Activities:** Take a guided tour of Monument Valley to see iconic formations such as the Mittens and Merrick Butte.
- **Evening:** Enjoy a sunset view and overnight stay in Monument Valley or nearby.

### Day 5: Monument Valley to Zion National Park
- **Morning:** Depart for Zion National Park (~4 hours, 210 miles).
- **Afternoon:** Arrive in Zion; explore the park's visitor center.
- **Activities:** Short hikes like Canyon Overlook or Riverside Walk.
- **Evening:** Overnight stay in Springdale, just outside Zion.

### Day 6: Explore Zion National Park
- **Morning and Afternoon:** Spend the day hiking and exploring Zion. Popular hikes include Angel's Landing (permit required) or The Narrows.
- **Evening:** Relax and enjoy your last evening in the park.

### Day 7: Zion to Las Vegas to Los Angeles
- **Morning:** Depart Zion for Las Vegas (~2.5 hours, 160 miles).
- **Activities:** Quick stop in Las Vegas for a late breakfast or early lunch, and explore the Strip or Fremont Street if time allows.
- **Afternoon:** Continue drive back to Los Angeles (~4 hours, 270 miles).
- **Evening:** Arrive back in Los Angeles.

### Tips:
- **Accommodations:** Book hotels or campsites in advance, especially in popular areas like Grand Canyon and Zion.
- **Permits:** Check for any required permits, especially for hikes like Angel's Landing.
- **Weather:** Be prepared for varying weather conditions, especially at higher elevations.
- **Fuel and Supplies:** Plan fuel stops and stock up on water and snacks, especially for longer stretches between destinations.

This itinerary provides a balance of driving and sightseeing, ensuring you have ample time to explore each destination while keeping the trip relaxed and enjoyable. Enjoy your adventure!

In [ ]:
# What a crazy drive for Day 7!

## 4\. Role-Playing Prompting  

This section creates a role-based interaction, where we assigning the AI the role of a professional travel planner. The system message helps guide the AI to generate detailed and engaging itineraries based on user requests.

The prompt includes:
- A system message to define the AI’s role and expertise.
- A human message with the user’s travel request.

In [7]:
# Role-playing prompt
system_prompt = """
You are a professional travel planner specializing in crafting detailed and engaging itineraries for travelers. 
"""

messages = [
        SystemMessage(content=system_prompt),
        HumanMessage(content=basic_prompt)
    ]

response2 = llm_gpt.invoke(messages)

display(Markdown(response2.content))

Planning a road trip through the Southwest is a fantastic way to explore the diverse landscapes and iconic landmarks of the region. Here’s a detailed 7-day itinerary that covers all the destinations you mentioned, starting and ending in Los Angeles:

### Day 1: Los Angeles to Kingman, AZ
- **Afternoon:** Depart Los Angeles after lunch. Head east on I-10, then take I-40 E towards Kingman, AZ. This drive will take approximately 5-6 hours.
- **Evening:** Arrive in Kingman, a charming Route 66 town. Check into a local hotel and enjoy dinner at a classic diner like Mr. D'z Route 66 Diner.

### Day 2: Kingman to Grand Canyon National Park
- **Morning:** Depart early for the Grand Canyon. Drive approximately 2.5 hours to the South Rim.
- **Afternoon:** Spend time exploring iconic viewpoints such as Mather Point and the Rim Trail. Grab lunch at the El Tovar Dining Room.
- **Evening:** Stay overnight at a lodge in the park or in nearby Tusayan.

### Day 3: Grand Canyon to Page, AZ
- **Morning:** Enjoy a sunrise walk along the rim or a short hike, then drive about 2.5 hours to Page.
- **Afternoon:** Visit Antelope Canyon (book a guided tour in advance) and stop by Horseshoe Bend for stunning views.
- **Evening:** Stay overnight in Page. Consider dining at Big John’s Texas BBQ for a hearty meal.

### Day 4: Page to Monument Valley
- **Morning:** Depart for Monument Valley, approximately a 2-hour drive. Along the way, enjoy the scenic views of the desert landscape.
- **Afternoon:** Explore Monument Valley with a Navajo-guided jeep tour to see iconic landmarks like the Mittens and John Ford’s Point.
- **Evening:** Stay at The View Hotel or nearby accommodations for an unforgettable night under the stars.

### Day 5: Monument Valley to Zion National Park
- **Morning:** Depart early for Zion National Park, a 4-5 hour drive.
- **Afternoon:** Arrive at Zion and check into your accommodation. Spend the afternoon hiking shorter trails like the Riverside Walk or exploring Zion Canyon.
- **Evening:** Stay overnight in Springdale, the gateway town to Zion. Enjoy dinner at a local restaurant like Oscar’s Café.

### Day 6: Zion National Park to Las Vegas
- **Morning:** Start your day with an early hike, such as the popular Angels Landing (permit required) or the Emerald Pools.
- **Afternoon:** Depart for Las Vegas, a 2.5-hour drive. Check into your hotel on the Strip.
- **Evening:** Enjoy the vibrant nightlife of Las Vegas. Consider a show, dining at a renowned restaurant, or simply strolling along the Strip.

### Day 7: Las Vegas to Los Angeles
- **Morning:** Spend the morning relaxing or doing a bit of last-minute sightseeing or shopping in Las Vegas.
- **Afternoon:** Depart for Los Angeles, a 4-5 hour drive. Aim to reach LA by mid-afternoon.

### Tips for Your Trip:
- **Accommodations:** Book your accommodations well in advance, especially for Grand Canyon and Zion National Park.
- **Permits/Reservations:** Secure permits for hikes like Angels Landing and tours for Antelope Canyon.
- **Travel Apps:** Use travel apps for navigation and check road conditions, as well as to find nearby dining options.

This itinerary allows you to experience the highlights of the Southwest while keeping travel times manageable. Enjoy your adventure!

## 5\. In-Context Learning  

This section explores in-context learning or few-short prompting, where we add structure, context, and clear expectations to both the system and user prompts. These refinements aims to make the model's responses more structured, relevant, and practical.

Key Improvements:
- System Prompt – Defines clear guidelines and specific requirements, ensuring well-organized, actionable itineraries. 
- User Prompt – Adds detailed trip preferences to produce a highly tailored and relevant plan.

Using few-shot learning with structured context results: 
- Better-structured itineraries – Includes daily schedules, travel times, dining options, accommodations, and travel tips.
- Improved context awareness – The AI considers trip priorities, driving limits, and sightseeing goals
- Greater consistency – Responses follow a clear format with well-defined expectations.


In [ ]:
# Add specific structural requirements and expectations.
refined_system_prompt = """
You are a professional travel planner specializing in crafting detailed and engaging itineraries for travelers.  
Your itineraries should include structured day-by-day plans with specific activities, travel times, meal recommendations, accommodation options, and insider travel tips.  
Ensure the itinerary is practical, well-paced, and optimized for an enjoyable experience while considering factors like drive times, sightseeing opportunities, and local recommendations.  

At the end of each itinerary, include a **"Travel Tips & Considerations"** section with general and destination-specific advice to enhance the travel experience. These should be practical, actionable tips that help travelers prepare better and avoid common pitfalls.  

### **Example of Travel Tips & Considerations:**  
To ensure a smooth and enjoyable trip, book accommodations in advance, especially in high-demand areas, and be mindful of local driving laws, including speed limits and parking restrictions. 
Limit daily driving to a reasonable duration to avoid fatigue and maximize sightseeing opportunities during daylight hours. 
Consider purchasing relevant travel passes, such as national park or city passes, to save on entry fees and streamline access to attractions. 
Plan meal stops ahead of time, particularly in remote areas with limited dining options, and always check the weather forecast to pack accordingly. 
Respect local customs, follow park regulations, and stay informed about any travel advisories or seasonal conditions that may affect your itinerary. 
"""

# Add specific structural requirements and expectations.
refined_user_prompt = """
I'm planning a 7-day Southwest road trip starting and ending in Los Angeles. We'll be leaving LA after lunch on the first day and need to return to LA in the afternoon on the last day. I want a well-paced itinerary that covers the Grand Canyon, Page, Monument Valley, Zion National Park, and Las Vegas. 
I'd like to keep driving times reasonable—preferably no more than six hours a day when possible. 
To make it a proper loop, I'd like to stop in Las Vegas twice—once at the start and again at the end.  

For the experience, I want to catch both sunrise and sunset at the Grand Canyon, with one morning and one afternoon being sufficient. 
I want to visit Antelope Canyon around midday, and see Horseshoe Bend at sunset. 
At Monument Valley, I'd love to drive the 17-mile loop to take in the scenery. 
And since I'll be back in Las Vegas toward the end of the trip, I'd like some time in the afternoon or evening to check out The Sphere for a show.  

I also want to try some great local food and stay at well-reviewed hotels along the way. 
Can you design an itinerary that balances sightseeing, good meals, and reasonable travel times?

"""

messages = [
        SystemMessage(content=refined_system_prompt),
        HumanMessage(content=refined_user_prompt)
    ]

response3 = llm_gpt.invoke(messages)

display(Markdown(response3.content))

Certainly! Here's a well-paced 7-day Southwest road trip itinerary that starts and ends in Los Angeles, covering all your requested destinations and experiences:

### **Day 1: Los Angeles to Las Vegas**
- **Afternoon:**
  - Depart Los Angeles around 1 PM.
  - **Drive Time:** Approximately 4.5 hours.
- **Evening:**
  - Arrive in Las Vegas by early evening.
  - Check into The Venetian Resort for a luxurious stay.
  - **Dinner:** Head to Carson Kitchen for modern American cuisine.
- **Activity:** Stroll along the Las Vegas Strip and catch the Bellagio Fountain show.

### **Day 2: Las Vegas to Grand Canyon (South Rim)**
- **Morning:**
  - Depart Las Vegas by 8 AM.
  - **Drive Time:** Approximately 4.5 hours to the Grand Canyon South Rim.
- **Afternoon:**
  - Check into Yavapai Lodge inside the park.
  - Spend the afternoon exploring the Visitor Center and Mather Point.
- **Evening:**
  - **Sunset:** Watch the sunset at Hopi Point.
  - **Dinner:** Enjoy dinner at the El Tovar Dining Room for classic comfort food.

### **Day 3: Grand Canyon to Page**
- **Early Morning:**
  - **Sunrise:** Catch the sunrise at Yaki Point.
  - Breakfast at your lodge.
- **Late Morning:**
  - Depart for Page.
  - **Drive Time:** Approximately 2.5 hours.
- **Afternoon:**
  - **Activity:** Tour Antelope Canyon around midday.
  - Check into Hyatt Place Page/Lake Powell.
- **Evening:**
  - **Sunset:** Visit Horseshoe Bend.
  - **Dinner:** Indulge in local flavors at Big John's Texas BBQ.

### **Day 4: Page to Monument Valley**
- **Morning:**
  - Depart for Monument Valley.
  - **Drive Time:** Approximately 2 hours.
- **Midday:**
  - Drive the 17-mile loop in Monument Valley.
  - Lunch picnic with scenic views.
- **Afternoon:**
  - Check into The View Hotel for unparalleled views of the valley.
- **Evening:**
  - **Dinner:** Enjoy Navajo cuisine at The View Restaurant.
  - Relax and stargaze from your hotel balcony.

### **Day 5: Monument Valley to Zion National Park**
- **Morning:**
  - Depart for Zion National Park.
  - **Drive Time:** Approximately 4.5 hours.
- **Afternoon:**
  - Check into Zion Lodge within the park for convenience.
  - Hike the Emerald Pools Trail or visit the Visitor Center.
- **Evening:**
  - **Dinner:** Enjoy a meal at the Zion Lodge's Red Rock Grill.
  - Relax by the fire pit at the lodge.

### **Day 6: Zion National Park to Las Vegas**
- **Morning:**
  - Early morning hike to Canyon Overlook for stunning views.
  - Depart for Las Vegas around 11 AM.
  - **Drive Time:** Approximately 2.5 hours.
- **Afternoon:**
  - Check into ARIA Resort & Casino.
  - Free time to relax or explore the pool area.
- **Evening:**
  - **Activity:** Attend a show at The Sphere.
  - **Dinner:** Dine at Momofuku for a unique culinary experience.

### **Day 7: Las Vegas to Los Angeles**
- **Morning:**
  - Leisurely breakfast at the hotel.
  - Depart for Los Angeles by 10 AM.
  - **Drive Time:** Approximately 4.5 hours.
- **Afternoon:**
  - Arrive in Los Angeles by mid-afternoon.

---

### **Travel Tips & Considerations**
- **Accommodation:** Book hotels in advance, especially those within national parks, to ensure availability.
- **Driving:** Keep an eye on the speed limits and road conditions. Consider using a GPS or offline maps app for navigation.
- **National Park Pass:** Purchase an America the Beautiful pass to save on entrance fees for multiple parks.
- **Packing:** Prepare for varying weather conditions, especially cooler temperatures in the morning and evening at higher elevations.
- **Dining:** Make restaurant reservations ahead of time when possible, especially in popular tourist areas.
- **Time Management:** Allow buffer time for unexpected delays. Always check opening hours for attractions and plan your visits accordingly.
- **Cultural Etiquette:** Respect Navajo Nation regulations and guidelines when visiting Monument Valley and other culturally significant sites.

## 6\. Chain of Thought Prompting  

This section experiments Chain of Thought prompting. This approach improves AI reasoning by guiding the model to think step by step when generating a response.

Key Improvements:
- The user prompt now includes a structured breakdown of logical steps for itinerary planning.
- Explicit instructions direct the AI to map routes, calculate driving times, consider sunrise/sunset, and optimize stops.


In [15]:
# Adds specific structural requirements and expectations
refined_cot_user_prompt = """
I'm planning a 7-day Southwest road trip starting and ending in Los Angeles. We'll be leaving LA after lunch on the first day and need to return to LA in the afternoon on the last day. I want a well-paced itinerary that covers the Grand Canyon, Page, Monument Valley, Zion National Park, and Las Vegas. 
I'd like to keep driving times reasonable—preferably no more than six hours a day when possible. 
To make it a proper loop, I'd like to stop in Las Vegas twice—once at the start and again at the end.  

For the experience, I want to catch both sunrise and sunset at the Grand Canyon, with one morning and one afternoon being sufficient. 
I want to visit Antelope Canyon around midday, and see Horseshoe Bend at sunset. 
At Monument Valley, I'd love to drive the 17-mile loop to take in the scenery. 
And since I'll be back in Las Vegas toward the end of the trip, I'd like some time in the afternoon or evening to check out The Sphere for a show.  

I also want to try some great local food and stay at well-reviewed hotels along the way.

Please help me design an itinerary by thinking through this step by step:
1. First, map out the logical route considering the loop nature of the trip and the requirement to visit Las Vegas twice
2. Then, calculate driving times between each stop to ensure they're reasonable
3. Factor in sunrise/sunset timings at specific locations (Grand Canyon, Horseshoe Bend)
4. Consider optimal times for attractions (like midday for Antelope Canyon)
5. Plan meal stops and hotel locations based on the daily schedule
6. Finally, compile everything into a day-by-day itinerary

Can you design an itinerary that balances sightseeing, good meals, and reasonable travel times?

"""

messages = [
        SystemMessage(content=refined_system_prompt),
        HumanMessage(content=refined_cot_user_prompt)
    ]

response4 = llm_gpt.invoke(messages)

display(Markdown(response4.content))

### Day 1: Los Angeles to Las Vegas
- **Afternoon**: Depart Los Angeles after lunch. Drive to Las Vegas (approx. 4.5 hours).
- **Evening**: Arrive in Las Vegas. Check into a hotel such as The Venetian Resort or Bellagio Hotel.
- **Dinner**: Enjoy a meal at Yardbird Southern Table & Bar for classic American comfort food.
- **Activity**: Explore the Las Vegas Strip and catch the Bellagio Fountains show.

### Day 2: Las Vegas to Grand Canyon (South Rim)
- **Morning**: Depart Las Vegas early (approx. 4.5 hours drive to the Grand Canyon South Rim).
- **Lunch**: Stop en route in Kingman, AZ, for a quick bite at Mr. D'z Route 66 Diner.
- **Afternoon**: Arrive at Grand Canyon National Park. Check into Yavapai Lodge or Maswik Lodge.
- **Sunset**: Head to Hopi Point or Yavapai Point for a stunning sunset view.
- **Dinner**: Enjoy dinner at the lodge or at the Bright Angel Restaurant.

### Day 3: Grand Canyon to Page
- **Sunrise**: Wake up early to catch the sunrise at Mather Point.
- **Morning**: Discover more viewpoints along the South Rim, like Desert View.
- **Lunch**: Have a meal at El Tovar Dining Room with its incredible views.
- **Afternoon**: Depart for Page, AZ (approx. 2.5 hours).
- **Evening**: Check into Hyatt Place Page/Lake Powell. 
- **Dinner**: Dine at Big John's Texas BBQ for a local experience.

### Day 4: Page to Monument Valley
- **Morning**: Tour Antelope Canyon (book a midday tour for optimal lighting).
- **Lunch**: Eat at Into the Grand for a taste of Navajo culture and cuisine.
- **Afternoon**: Visit Horseshoe Bend; arrive at least an hour before sunset to enjoy the view.
- **Evening**: Drive to Monument Valley (approx. 2 hours). 
- **Accommodation**: Stay at The View Hotel for its panoramic views.
- **Dinner**: Dine at The View Restaurant within the hotel.

### Day 5: Monument Valley to Zion National Park
- **Morning**: Drive the 17-mile loop road in Monument Valley (allow 2-3 hours).
- **Lunch**: Enjoy a packed lunch or stop at Goulding’s Stagecoach Dining Room.
- **Afternoon**: Drive to Zion National Park (approx. 4 hours).
- **Evening**: Check into Zion Lodge inside the park or nearby Springdale hotels like Cliffrose Lodge.
- **Dinner**: Try Oscar's Cafe in Springdale for a hearty meal.

### Day 6: Zion National Park to Las Vegas
- **Morning**: Hike one of Zion’s famous trails, like the Canyon Overlook Trail.
- **Lunch**: Have a meal at the Red Rock Grill in Zion Lodge.
- **Afternoon**: Depart for Las Vegas (approx. 2.5 hours).
- **Evening**: Check into the same or a different hotel like ARIA Resort & Casino.
- **Activity**: Attend a show at The Sphere.
- **Dinner**: Enjoy a fine dining experience at Raku or Lotus of Siam for authentic Thai cuisine.

### Day 7: Las Vegas to Los Angeles
- **Morning**: Leisurely breakfast at Bouchon Bistro.
- **Late Morning**: Explore any last-minute attractions or shopping.
- **Afternoon**: Depart for Los Angeles (approx. 4.5 hours).
- **Evening**: Arrive in Los Angeles.

### Travel Tips & Considerations:
- **Accommodations**: Book all accommodations in advance, especially in high-demand areas like the Grand Canyon and Monument Valley.
- **Driving**: Stick to the speed limits and be prepared for long stretches without services, especially in rural areas.
- **National Park Pass**: Consider purchasing an America the Beautiful pass if visiting multiple national parks.
- **Weather**: Check forecasts and pack accordingly, as temperatures can vary widely, especially at higher elevations.
- **Tours**: Book Antelope Canyon tours well in advance and arrive early to your booking.
- **Time Zones**: Be mindful of time changes, especially between Arizona and Utah, as some areas do not observe Daylight Saving Time.
- **Sunrise/Sunset**: Verify the exact timings for sunrise and sunset as they vary by season.
- **Cultural Respect**: Respect Navajo Nation rules and customs, particularly in Monument Valley and Antelope Canyon.

## 7\. Chain of Thought with DeepSeek-R1

This approach leverages DeepSeek-R1’s built-in reasoning abilities without requiring step-by-step instructions in the prompt.

Key Benefits:
- The reasoning-optimized model naturally follows a logical thought process.
- It allows us to observe the AI’s reasoning, offering insight into how it makes travel planning decisions.

In [9]:
# CoT using deepseek-r1

messages = [
        SystemMessage(content=refined_system_prompt),
        HumanMessage(content=refined_user_prompt) # without adding the step by step instructions
    ]

response5 = llm_deepseek_r1.invoke(messages)

display(Markdown(response5.content))

<think>
Okay, let's tackle this Southwest road trip itinerary. The user wants a 7-day loop starting and ending in LA, hitting Grand Canyon, Page, Monument Valley, Zion, and Las Vegas twice. They mentioned keeping driving times under six hours where possible. Let me start by mapping out the route.

First day: Leave LA after lunch. The drive to Las Vegas is about 4 hours, so that's manageable. They want to stop in Vegas at the start and end. So Day 1 ends in Vegas. Then Day 2, drive to Grand Canyon. The drive from Vegas to Grand Canyon South Rim is around 4.5 hours. They want sunrise and sunset there. Maybe spend Day 2 evening and Day 3 morning at Grand Canyon. Then Day 3 drive to Page, which is about 2.5 hours. That allows for Antelope Canyon midday and Horseshoe Bend sunset. 

Next, Monument Valley. From Page to Monument Valley is around 2 hours. They want the 17-mile loop, so maybe a half-day there. Then drive to Zion, which is about 4.5 hours. That's within the six-hour limit. Zion would need a full day, so Day 5 and 6? Wait, let's check the days.

Wait, total days are 7. Let's count:

Day 1: LA to Vegas
Day 2: Vegas to Grand Canyon
Day 3: Grand Canyon to Page
Day 4: Page to Monument Valley
Day 5: Monument Valley to Zion
Day 6: Zion to Vegas
Day 7: Vegas to LA

Wait, that's 7 days. But the user wants to return to LA on the last day (Day 7) afternoon. So Day 7 would be Vegas to LA, which is 4 hours. That works. 

Now, activities: Grand Canyon sunrise and sunset. So on Day 2, arrive Grand Canyon in the afternoon, catch sunset. Then sunrise the next morning. Then drive to Page. Antelope Canyon midday on Day 3, Horseshoe Bend sunset. Then Day 4, drive to Monument Valley, do the loop. Then Day 5 drive to Zion, spend the afternoon there. Day 6 explore Zion, then drive to Vegas. Day 7 Vegas to LA.

Wait, but Zion might need more time. Maybe Day 5 is Monument Valley to Zion (4.5 hours), then Day 6 full day in Zion, then Day 7 Zion to Vegas (2.5 hours) and then Vegas to LA (4 hours). But the user wants to end in Vegas on the last day, but needs to return to LA. So Day 7: Zion to Vegas in the morning, then Vegas to LA in the afternoon. That might be tight. Let me check driving times.

Zion to Vegas is about 2.5 hours. Then Vegas to LA is 4 hours. So total driving on Day 7 would be 6.5 hours. But the user prefers no more than six hours. Hmm. Maybe adjust. Alternatively, on Day 6, after Zion, drive to Vegas in the evening. Then Day 7 is just Vegas to LA. That way, Day 6 has Zion in the morning, drive to Vegas (2.5 hours), arrive in the afternoon, have time for The Sphere show. Then Day 7 is Vegas to LA in the afternoon. That might work better.

Let me adjust the days:

Day 1: LA to Vegas
Day 2: Vegas to Grand Canyon (4.5h)
Day 3: Grand Canyon to Page (2.5h)
Day 4: Page to Monument Valley (2h)
Day 5: Monument Valley to Zion (4.5h)
Day 6: Zion, then drive to Vegas (2.5h)
Day 7: Vegas to LA (4h)

This way, Day 6 ends in Vegas, allowing for The Sphere show that evening. Then Day 7 is just the drive back. That keeps driving times under six hours each day.

Now, activities: Grand Canyon sunset on Day 2, sunrise on Day 3. Antelope Canyon midday on Day 3? Wait, Day 3 is Grand Canyon to Page. Arrive in Page by late morning, do Antelope Canyon tour (which usually requires booking in advance), then Horseshoe Bend sunset. That works.

Monument Valley's 17-mile loop on Day 4. Then drive to Zion on Day 5, arriving afternoon, maybe some hiking. Day 6 full day in Zion, then drive to Vegas in the evening. Day 7 Vegas to LA.

Accommodations: Need well-reviewed hotels in each location. For Grand Canyon, maybe El Tovar or Bright Angel Lodge. Page has Lake Powell Resort. Monument Valley: The View Hotel. Zion: Zion Lodge or Springdale hotels. Vegas: Bellagio or similar.

Meals: Local spots. In Page, maybe State 48 Tavern. Monument Valley: Goulding's Lodge. Zion: Oscar's Cafe. Vegas: Eiffel Tower Restaurant.

Travel tips: Book Antelope Canyon tours early, check for timed entries. Monument Valley loop may require a guide if not self-driving. Zion shuttle system. Vegas Sphere tickets in advance.

Also, driving times need to be accurate. Let me verify:

- LA to Vegas: 4h
- Vegas to Grand Canyon South Rim: 4.5h
- Grand Canyon to Page: 2.5h
- Page to Monument Valley: 2h
- Monument Valley to Zion: 4.5h
- Zion to Vegas: 2.5h
- Vegas to LA: 4h

Total driving times per day are within limits except maybe Day 7, but that's just 4h. Wait, Day 7 is Vegas to LA, which is 4h, so the user can leave in the morning and arrive in LA by afternoon. Perfect.

This should fit. Now structure the itinerary day by day with activities, meals, hotels, and tips.
</think>

Here’s a **7-day Southwest road trip itinerary** optimized for scenic drives, iconic sights, and minimal fatigue. Hotels and meals are chosen for quality, local flavor, and convenience.

---

### **Day 1: Los Angeles → Las Vegas**  
**Drive**: 4 hours (leave LA after lunch)  
**Stay**: *Bellagio* (iconic Strip location) or *The Cosmopolitan* (modern luxury)  
**Evening**:  
- Stroll the Strip, visit Bellagio Fountains  
- Dinner at *Bazaar Meat by José Andrés* (creative Spanish-inspired cuisine)  

---

### **Day 2: Las Vegas → Grand Canyon (South Rim)**  
**Drive**: 4.5 hours (depart early)  
**Stay**: *El Tovar Hotel* (historic lodge) or *Bright Angel Lodge* (rustic charm)  
**Activities**:  
- Arrive by 3 PM, hike *South Kaibab Trail* to Ooh Aah Point  
- **Sunset**: Hopi Point (shuttle access)  
**Dinner**: *El Tovar Dining Room* (Southwestern fare with canyon views)  

---

### **Day 3: Grand Canyon → Page, AZ**  
**Drive**: 2.5 hours (leave after sunrise)  
**Stay**: *Lake Powell Resort* (lake views) or *Hyatt Place Page* (modern comfort)  
**Morning**:  
- **Sunrise**: Mather Point or Yavapai Point  
- Quick breakfast at *Bright Angel Coffee House*  
**Afternoon**:  
- **Antelope Canyon Tour** (book Upper or Lower Canyon midday for best light)  
**Evening**:  
- **Sunset**: Horseshoe Bend (10-min drive from Page; arrive 1 hour early)  
**Dinner**: *State 48 Tavern* (craft burgers + local beer)  

---

### **Day 4: Page → Monument Valley**  
**Drive**: 2 hours (scenic US-163)  
**Stay**: *The View Hotel* (only lodging inside Monument Valley; book a balcony room)  
**Activities**:  
- **Monument Valley 17-Mile Scenic Drive** (self-guided or Navajo-guided tour)  
- Stop at John Ford’s Point and Totem Pole formations  
**Dinner**: *Goulding’s Lodge Restaurant* (Navajo tacos + fry bread)  

---

### **Day 5: Monument Valley → Zion National Park**  
**Drive**: 4.5 hours (via Kayenta and UT-59)  
**Stay**: *Zion Lodge* (in-park) or *Cliffrose Springdale* (luxury with river views)  
**Afternoon**:  
- Hike *Riverside Walk* (easy) or *Emerald Pools Trail* (moderate)  
**Dinner**: *Oscar’s Cafe* (hearty Southwestern plates in Springdale)  

---

### **Day 6: Zion → Las Vegas**  
**Morning**:  
- **Sunrise Hike**: *Angels Landing* (permits required) or *Canyon Overlook Trail* (easy)  
**Drive**: 2.5 hours to Vegas (leave Zion by 2 PM)  
**Stay**: *Wynn Las Vegas* (luxury) or *NoMad Hotel* (boutique vibe)  
**Evening**:  
- **The Sphere Experience**: Book a show like *Postcard from Earth* (arrive early)  
- Dinner at *Brezza* (Italian seafood) or *Eiffel Tower Restaurant* (romantic Strip views)  

---

### **Day 7: Las Vegas → Los Angeles**  
**Drive**: 4 hours (leave Vegas by 10 AM for a 2 PM LA arrival)  
**Optional Stop**: *Peggy Sue’s 50s Diner* (near Barstow) for retro milkshakes  

---

### **Travel Tips & Considerations**  
1. **Antelope Canyon**: Book tours 2-3 months in advance (Upper Canyon for light beams, Lower Canyon for fewer crowds).  
2. **Monument Valley**: The 17-mile loop is rough but drivable in most cars; guided tours add cultural insights.  
3. **Zion Shuttles**: Required for scenic canyon hikes March–November; park at Springdale Visitor Center.  
4. **Grand Canyon Parking**: Arrive early for sunrise spots; use free shuttles to avoid congestion.  
5. **Las Vegas**: Reserve Sphere tickets early (dates sell out fast); valet parking saves time.  
6. **Weather**: Layer clothing—desert temps swing from 40s°F (mornings) to 80s°F (days).  
7. **Food**: Pack snacks/water for remote drives (e.g., Page to Monument Valley).  

--- 

This balances iconic sights, manageable drives, and unique local flavors. Let me know if you’d like adjustments! 🌄🚗

## 8\. ReAct Prompting

Now let's expreiment ReAct prompting. This approach integrates real-time web search, enabling the AI to fetch the latest travel details, including weather, seasonal conditions, and updated recommendations.

Using the Reasoning + Action (ReAct) framework, the AI:
- Analyzes the request to understand user needs.
- Searches for relevant information when necessary.
- Generates a structured itinerary with accurate, up-to-date recommendations.

**Note on LangChain Updates**

LangChain is gradually replacing AgentExecutor with LangGraph, a more flexible and scalable framework for agent workflows. While this method remains functional, adopting LangGraph is recommended for long-term support and enhanced control.  

*References:*
- [LangChain Agents](https://python.langchain.com/docs/concepts/agents/)
- [Migrating from AgentExecutor to LangGraph](https://python.langchain.com/docs/how_to/migrate_agent/)


In [16]:
# Tool calling agent or ReAct agent
refined_system_prompt_with_tool = """
You are a professional travel planner specializing in crafting detailed and engaging itineraries for travelers.  
Your itineraries should include structured day-by-day plans with specific activities, travel times, meal recommendations, accommodation options, and insider travel tips.  
Ensure the itinerary is practical, well-paced, and optimized for an enjoyable experience while considering factors like drive times, sightseeing opportunities, and local recommendations.  

Search the web for information if you need to. For example, you can search the weather and season of the destinations.

At the end of each itinerary, include a **"Travel Tips & Considerations"** section with general and destination-specific advice to enhance the travel experience. These should be practical, actionable tips that help travelers prepare better and avoid common pitfalls.  

### **Example of Travel Tips & Considerations:**  
To ensure a smooth and enjoyable trip, book accommodations in advance, especially in high-demand areas, and be mindful of local driving laws, including speed limits and parking restrictions. 
Limit daily driving to a reasonable duration to avoid fatigue and maximize sightseeing opportunities during daylight hours. 
Consider purchasing relevant travel passes, such as national park or city passes, to save on entry fees and streamline access to attractions. 
Plan meal stops ahead of time, particularly in remote areas with limited dining options, and always check the weather forecast to pack accordingly. 
Respect local customs, follow park regulations, and stay informed about any travel advisories or seasonal conditions that may affect your itinerary. 
"""
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", f"{refined_system_prompt_with_tool}"), # System message defining agent's role
        ("human", "{input}"), # Placeholder for user-provided input
        ("ai", "{agent_scratchpad}")  # Required for agent memory and processing
    ]
)

# Define a web search tool using Tavily API to fetch relevant travel information
search_tool = TavilySearchResults(
    max_results=3,
    search_depth="advanced", # Uses advanced search depth for more accurate results
    include_answer=True, # Include a short answer to original query in the search results. 
    tavily_api_key= tavily_api_key # You have defined this API key in the .env file. 
)

# List of tools available to the agent (currently only the search tool)
tools = [search_tool]

# Create an AI agent that uses external tools (e.g., the search tool defined earlier).
agent = create_tool_calling_agent(llm_gpt, tools, prompt)

# Initialize the agent that is using the tools.
agent_executor = AgentExecutor(agent=agent, tools=tools)

# Invoke the agent with a user-defined input prompt.
response6 = agent_executor.invoke({"input": f'{refined_cot_user_prompt}?'})

In [17]:
display(Markdown(response6['output']))

Based on the driving times between the destinations and your preferences, here is a well-paced 7-day itinerary for your Southwest road trip:

### **Day 1: Los Angeles to Las Vegas**
- **Departure:** Leave Los Angeles after lunch around 1:00 PM.
- **Drive Time:** Approximately 4-5 hours.
- **Arrival:** Reach Las Vegas by 6:00 PM.
- **Evening:** Explore the Las Vegas Strip, have dinner at a popular local restaurant like "Lotus of Siam" for authentic Thai cuisine.
- **Accommodation:** Stay at a well-reviewed hotel such as "The Venetian" or "Bellagio."

### **Day 2: Las Vegas to Grand Canyon South Rim**
- **Departure:** Early morning departure around 7:00 AM.
- **Drive Time:** Approximately 4.5-5 hours.
- **Afternoon:** Arrive at the Grand Canyon South Rim by noon. Explore the rim and enjoy lunch at "El Tovar Dining Room."
- **Evening:** Catch the sunset at the South Rim. 
- **Accommodation:** Stay overnight at "Yavapai Lodge" or "El Tovar Hotel" inside the park.

### **Day 3: Grand Canyon to Page, AZ**
- **Morning:** Catch the sunrise at the Grand Canyon, then enjoy breakfast.
- **Departure:** Leave for Page around 9:00 AM.
- **Drive Time:** Approximately 2.5 hours.
- **Midday:** Arrive in Page and visit Antelope Canyon around noon for optimal lighting.
- **Evening:** Visit Horseshoe Bend at sunset.
- **Accommodation:** Stay at "Lake Powell Resort" or "Hyatt Place Page/Lake Powell."

### **Day 4: Page to Monument Valley**
- **Morning:** Enjoy a leisurely breakfast in Page.
- **Departure:** Head to Monument Valley around 10:00 AM.
- **Drive Time:** Approximately 2 hours.
- **Afternoon:** Drive the 17-mile loop at Monument Valley, taking in the stunning red rock formations.
- **Evening:** Dinner at "The View Restaurant" with panoramic views of the valley.
- **Accommodation:** Stay at "The View Hotel" in Monument Valley.

### **Day 5: Monument Valley to Zion National Park**
- **Departure:** Early start around 8:00 AM.
- **Drive Time:** Approximately 4.5 hours.
- **Afternoon:** Arrive at Zion and explore some of the park’s trails, like the Riverside Walk.
- **Evening:** Casual dinner in Springdale, UT at "Oscar's Cafe."
- **Accommodation:** Stay at "Zion Lodge" inside the park or "Cliffrose Springdale."

### **Day 6: Zion National Park to Las Vegas**
- **Morning:** Hike the famous "Angels Landing" or opt for a less strenuous trail.
- **Departure:** Leave Zion in the early afternoon around 2:00 PM.
- **Drive Time:** Approximately 2.5 hours.
- **Evening:** Arrive in Las Vegas. Attend a show at The Sphere in the evening.
- **Accommodation:** Stay at "Aria Resort & Casino" or "Wynn Las Vegas."

### **Day 7: Las Vegas to Los Angeles**
- **Morning:** Relax in Las Vegas or do some last-minute sightseeing.
- **Departure:** Head back to Los Angeles after an early lunch around 11:00 AM.
- **Drive Time:** Approximately 4-5 hours.
- **Afternoon:** Arrive in Los Angeles by 4:00 PM.

### **Travel Tips & Considerations:**
- **Reservations:** Book accommodations and tours (especially for Antelope Canyon) in advance.
- **Driving:** Stay mindful of speed limits and driving conditions, especially in rural areas.
- **Park Regulations:** Follow park rules and guidelines, and respect local wildlife.
- **Weather:** Check the weather forecast for all destinations and pack accordingly.
- **Local Cuisine:** Explore local dining options for a more authentic experience.
- **Timing:** Account for time zone changes and adjust your schedule as needed.
- **Safety:** Keep hydrated and carry snacks, especially while hiking and on long drives. 

This itinerary offers a balanced mix of sightseeing, relaxation, and cultural experiences while keeping daily drive times manageable. Enjoy your road trip!

## 9\. Human Evaluation  

To assess the quality of AI-generated itineraries, compare them against the benchmark travel plan. 

Condiser using the following Evaluation Criteria:
- Accuracy & Completeness – Does the AI itinerary cover all requested destinations and key attractions?  
- Logical Flow & Feasibility – Are the travel routes, driving times, and sightseeing schedules practical?  
- Experience Optimization – Does it include well-timed stops for sunrise/sunset, meal breaks, and local highlights?  
- Realism & Practicality – Are the recommendations actionable, with reasonable pacing and accommodations?  